# Thu thập dữ liệu

## Giới thiệu
Dự án xây dựng mô hình dự đoán giá ô tô thông qua dữ liệu thực tế lấy từ các website mua bán xe trực tuyến. Notebook này tóm tắt quy trình lấy dữ liệu từ hai trang mua bán lớn của Việt Nam và chuẩn hóa chúng cho việc mô hình hóa.

## Mục tiêu
- Thu thập các thông số kỹ thuật (features) chính xác từ mỗi website.
- Ghi nhận nguồn gốc dữ liệu để so sánh nhanh giữa hai trang.

## Phạm vi
- Nguồn: Chotot (API) và Bonbanh (web scraping).
- Dữ liệu: Tiêu đề, giá, thông số kỹ thuật, vị trí, người bán, số ảnh (nếu có).
- Kết quả: CSV trong `datasets/` phục vụ phân tích và huấn luyện mô hình.

In [ ]:
import requests 
from bs4 import BeautifulSoup  
import pandas as pd  
import re  

N = 100  # Số lượng mẫu cần thu thập từ mỗi trang web

## Phương pháp

### 1. Thu thập dữ liệu
- **Chotot**: Dùng API nội bộ để lấy ID và sinh URL rao bán xe. Sau đó lấy thông tin chi tiết của xe từ URL đã sinh.
- **Bonbanh**: Đọc trang `bonbanh.com/oto` với phân trang để scrape danh sách listing, sau đó truy cập từng URL chi tiết.

### 2. Tối ưu hiệu suất
- Sử dụng **đa luồng** (threading) để trích xuất thông tin từ nhiều URL song song, giảm thời gian chờ đợi.
- Mỗi luồng xử lý một URL độc lập, kết quả được gom lại sau khi tất cả luồng hoàn thành.

### 3. Trích xuất features
#### Chotot
- Tiêu đề/giá nằm ở các tag rõ ràng (`<h1>` và `<b class="p26z2wb">`).
- Tất cả features được gom trong `div.p1ja3eq0`; cặp `span.bwq0cbs` đầu là label, sau là giá trị.
- Đọc vị trí, người bán, mô tả để đánh dấu dữ liệu có đủ thông tin.

#### Bonbanh
- Tiêu đề nằm trong phần `<h1>`.
- Giá/ hãng/ tên xe nằm trong tiêu đề
- Thông số kỹ thuật ở các `div.row` với `label` thể hiện tên, `span.inp` chứa giá trị.
- Lọc chỉ giữ các keys quan trọng (Năm sản xuất, Tình trạng, Hộp số, v.v.) để dễ chuẩn hóa.

In [13]:
def extract_chotot_features(url):
    """
    Trích xuất features chi tiết từ trang listing Chotot.
    Trả về dict với các label rõ ràng để merge sau này.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        features = {}  
        
        # Tiêu đề
        title_elem = soup.find('h1')
        if title_elem:
            features['title'] = title_elem.get_text(strip=True)
        
        # Giá
        price_elem = soup.find('b', class_='p26z2wb')
        if price_elem:
            features['price'] = price_elem.get_text(strip=True)
        
        # Features từ div.p1ja3eq0 (label trước, value sau)
        feature_divs = soup.find_all('div', class_='p1ja3eq0')
        for div in feature_divs:
            spans = div.find_all('span', class_='bwq0cbs')
            if len(spans) >= 2:
                key = spans[0].get_text(strip=True)
                value = spans[1].get_text(strip=True)
                if key and value:
                    features[key] = value
        
        # Vị trí 
        location_elem = soup.find('span', class_='bwq0cbs', string=re.compile(r'Quận|Phường|Huyện|Tp|Tỉnh'))
        if location_elem:
            features['location'] = location_elem.get_text(strip=True)
        
        # Người bán
        seller_link = soup.find('a', href=re.compile(r'/user/'))
        if seller_link:
            seller_name = seller_link.find('b')
            if seller_name:
                features['seller'] = seller_name.get_text(strip=True)
        
        # Mô tả 
        desc_elem = soup.find('div', class_='des_txt')
        if desc_elem:
            features['description'] = desc_elem.get_text(strip=True)
        
        return features
    except Exception as e:
        print(f"Lỗi khi trích xuất từ {url}: {e}")
        return {}


### Giải thích hàm trích xuất Chotot
Hàm này giả lập một trình duyệt nhẹ, tải về HTML chi tiết của listing, rồi lần lượt gom các cặp nhãn và giá trị từ `div.p1ja3eq0` (div chứa các cặp nhãn và giá trị) vào một dictionary. Những trường quan trọng như vị trí, người bán, mô tả hay số ảnh cũng được giữ lại để dễ đánh giá độ đầy đủ trước khi đẩy toàn bộ sang CSV.

#### Bonbanh
- Tiêu đề xuất hiện trong `<h1>`.
- Giá, tên hãng, tên xe nằm trong tiêu đề.
- Các thông số kỹ thuật được phân thành `div.row` (các dòng), mỗi cặp `<label>` và `<span class="inp">` tương ứng với tên và giá trị của thuộc tính.
- Lọc chỉ giữ những thuộc tính cốt lõi như Năm sản xuất, Tình trạng, Hộp số... để bảng dữ liệu không bị loãng với thông tin thừa.
- Sau cùng, ghi thêm vị trí và người bán để liên kết ngược tới nguồn.

In [14]:
def extract_bonbanh_features(url):
    """
    Trích xuất features từ URL Bonbanh.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        features = {}
        
        # Tiêu đề
        title_elem = soup.find('h1')
        if title_elem:
            features['title'] = title_elem.get_text(strip=True)
    
        # Các thông số kỹ thuật
        desired_keys = [
            'Năm sản xuất',
            'Tình trạng',
            'Số Km đã đi',
            'Xuất xứ',
            'Kiểu dáng',
            'Hộp số',
            'Động cơ',
            'Màu ngoại thất',
            'Màu nội thất',
            'Số chỗ ngồi',
            'Số cửa',
            'Dẫn động'
        ]
        rows = soup.find_all('div', class_='row')
        for row in rows:
            label_elem = row.find('label')
            inp_elem = row.find('span', class_='inp')
            if label_elem and inp_elem:
                key = label_elem.get_text(strip=True).replace(':', '')
                value = inp_elem.get_text(strip=True)
                if key in desired_keys and value:
                    features[key] = value
        
        # Vị trí + người bán 
        features['location'] = 'Hà Nội'
        seller_elem = soup.find('a', class_='cname')
        if seller_elem:
            features['seller'] = seller_elem.get_text(strip=True)
        
        return features
    except Exception as e:
        print(f"Lỗi khi trích xuất từ {url}: {e}")
        return {}


### Giải thích hàm trích xuất Bonbanh
Chúng ta giữ nguyên tiêu đề như cách người dùng gõ lên trang, chỉ dọn phần head/tail để tránh khoảng trắng dư. Sau khi trang đầy đủ được tải về, danh sách `div.row` cung cấp cặp `label`/`span.inp` nên hàm chỉ nhặt ra những thuộc tính cốt lõi (năm sản xuất, hộp số, màu sắc...) để tránh ghi quá nhiều thông tin thừa. Cuối cùng, location và seller được ghi vào để giữ nguồn gốc dữ liệu.

### 3. Triển khai Thu thập URL
Trước khi trích xuất feature, chúng ta cần danh sách URL cụ thể. Chotot cung cấp API giúp tránh việc iterate qua từng trang tìm kiếm; Bonbanh không có API công khai nên phải scrape danh sách car-item.

#### Thu thập URL từ Chotot và Bonbanh
- Chotot: gọi API `gateway.chotot.com/v1/public/ad-listing` với tham số limit để lấy top listings.
- Bonbanh: tải trang `https://bonbanh.com/oto`, duyệt `li.car-item`, gom tag `<a>` vào danh sách URL.

In [15]:
def collect_chotot_urls(n=10):
    """
    Thu thập N URL đầu từ trang mua bán ô tô trên chotot.com
    bằng cách gọi API gateway với pagination.
    """
    base_url = "https://gateway.chotot.com/v1/public/ad-listing"
    urls = []
    page = 0
    while len(urls) < n:
        params = {
            'w': 1,          # web flag
            'limit': min(50, n - len(urls)),  # max 50 per request
            'st': 's,k',     # sort by score, keyword
            'f': 'p',        # filter published
            'cg': 2010,      # category: mua bán ô tô
            'o': page * 50,  # offset
            'page': page     # page
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            ads = data.get('ads', [])
            if not ads:
                break  # no more ads
            
            for ad in ads:
                list_id = ad.get('list_id')
                if list_id:
                    url = f"https://xe.chotot.com/mua-ban-oto/{list_id}.htm"
                    urls.append(url)
                    if len(urls) >= n:
                        break
            
            page += 1
        except requests.RequestException as e:
            print(f"Lỗi khi gọi API trang {page}: {e}")
            break
    
    return urls[:n]

# Thu thập N URL đầu
chotot_urls = collect_chotot_urls(N)
print(f"Đã thu thập {len(chotot_urls)} URL từ Chotot:")
for url in chotot_urls:
    print(url)

Đã thu thập 100 URL từ Chotot:
https://xe.chotot.com/mua-ban-oto/129173883.htm
https://xe.chotot.com/mua-ban-oto/129237179.htm
https://xe.chotot.com/mua-ban-oto/128468488.htm
https://xe.chotot.com/mua-ban-oto/129237062.htm
https://xe.chotot.com/mua-ban-oto/129111941.htm
https://xe.chotot.com/mua-ban-oto/129236899.htm
https://xe.chotot.com/mua-ban-oto/128674965.htm
https://xe.chotot.com/mua-ban-oto/129236758.htm
https://xe.chotot.com/mua-ban-oto/129216579.htm
https://xe.chotot.com/mua-ban-oto/128530935.htm
https://xe.chotot.com/mua-ban-oto/129095292.htm
https://xe.chotot.com/mua-ban-oto/129090815.htm
https://xe.chotot.com/mua-ban-oto/129236270.htm
https://xe.chotot.com/mua-ban-oto/128202638.htm
https://xe.chotot.com/mua-ban-oto/128923637.htm
https://xe.chotot.com/mua-ban-oto/125136946.htm
https://xe.chotot.com/mua-ban-oto/129058770.htm
https://xe.chotot.com/mua-ban-oto/129092295.htm
https://xe.chotot.com/mua-ban-oto/129235574.htm
https://xe.chotot.com/mua-ban-oto/129235539.htm
https://x

### Giải thích hàm collect_chotot_urls
API Chotot giới hạn tối đa 50 kết quả mỗi lần gọi, vì vậy để lấy nhiều hơn, ta phải dùng phân trang (pagination). Hàm này lặp qua các trang bằng cách tăng `offset` và `page`, gọi API liên tiếp cho đến khi đủ `n` URL hoặc hết dữ liệu. Mỗi `list_id` từ API được ghép thành URL hoàn chỉnh để truy cập chi tiết listing.

In [16]:
def collect_bonbanh_urls(n=10):
    """
    Thu thập N URL đầu từ trang mua bán ô tô cũ trên bonbanh.com
    bằng cách scrape trang tìm kiếm với nhiều trang.
    """
    urls = []
    page = 1
    while len(urls) < n:
        try:
            url = f"https://bonbanh.com/oto/page,{page}"
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Tìm car listings trong li.car-item
            listings = soup.find_all('li', class_=lambda x: x and 'car-item' in x)
            if not listings:
                break  # no more listings
            
            for li in listings:
                a_tag = li.find('a', href=True)
                if a_tag:
                    href = a_tag['href']
                    if href.startswith('/'):
                        full_url = f"https://bonbanh.com{href}"
                    else:
                        full_url = f"https://bonbanh.com/{href}"
                    if full_url not in urls:
                        urls.append(full_url)
                        if len(urls) >= n:
                            break
            
            page += 1
        except requests.RequestException as e:
            print(f"Lỗi khi scrape trang {page}: {e}")
            break
    
    return urls[:n]

# Thu thập N URL đầu từ Bonbanh
bonbanh_urls = collect_bonbanh_urls(N)
print(f"Đã thu thập {len(bonbanh_urls)} URL từ Bonbanh:")
for url in bonbanh_urls:
    print(url)

Đã thu thập 100 URL từ Bonbanh:
https://bonbanh.com/xe-toyota-wigo-g-1.2-at-2025-6150510
https://bonbanh.com/xe-toyota-innova-2.0e-2016-6535576
https://bonbanh.com/xe-kia-carens-1.5d-premium-2022-6523189
https://bonbanh.com/xe-nissan-terra-s-2.5-mt-2wd-2018-6046907
https://bonbanh.com/xe-mazda-3-1.5l-luxury-2025-6533779
https://bonbanh.com/xe-mitsubishi-pajero-sport-2.4d-4x2-mt-2019-6535047
https://bonbanh.com/xe-mercedes_benz-glc-200-2020-6521266
https://bonbanh.com/xe-toyota-sienna-limited-3.5-2016-6503341
https://bonbanh.com/xe-kia-morning-x-line-2021-6539503
https://bonbanh.com/xe-toyota-vios-g-1.5-cvt-2025-6525945
https://bonbanh.com/xe-ford-territory-trend-1.5-at-2025-6504124
https://bonbanh.com/xe-mercedes_benz-c_class-c300-amg-2019-6461653
https://bonbanh.com/xe-volvo-xc40-t5-awd-r-design-2019-6235585
https://bonbanh.com/xe-toyota-land_cruiser-3.5-v6-2025-5663074
https://bonbanh.com/xe-mercedes_benz-s_class-s400l-2014-6528405
https://bonbanh.com/xe-mercedes_benz-glc-200-2021-65

### Giải thích hàm collect_bonbanh_urls
Trang Bonbanh hiển thị khoảng 20 listing trên mỗi trang, nên để thu thập đủ `n` URL, ta cần scrape nhiều trang liên tiếp. Hàm duyệt qua các trang (`page,1`, `page,2`...), tìm các `li.car-item`, lấy href và chuyển sang URL tuyệt đối. Quá trình dừng khi đủ số lượng hoặc khi trang không còn listing nào.

### 4. Triển khai Trích xuất và Lưu Dữ liệu với Đa luồng
Thay vì trích xuất tuần tự từng URL (chậm vì phải chờ mỗi request), ta dùng `ThreadPoolExecutor` để chạy song song nhiều request cùng lúc. Mỗi luồng gọi hàm trích xuất cho một URL, kết quả được gom lại khi tất cả luồng hoàn thành. Điều này giảm đáng kể thời gian xử lý khi có nhiều URL.

In [17]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# I. Trích xuất dữ liệu từ Chotot 
print("Bắt đầu trích xuất dữ liệu Chotot...")
chotot_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(extract_chotot_features, url): url for url in chotot_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            features = future.result()
            if features:
                chotot_data.append(features)
        except Exception as e:
            print(f"Lỗi với URL Chotot {url}: {e}")

df_chotot = pd.DataFrame(chotot_data)
df_chotot.to_csv('../datasets/raw_chotot_car_features.csv', index=False)
print(f"Chotot: Đã lưu {len(chotot_data)} bản ghi vào datasets/raw_chotot_car_features.csv")

# II. Trích xuất dữ liệu từ Bonbanh 
print("Bắt đầu trích xuất dữ liệu Bonbanh...")
bonbanh_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(extract_bonbanh_features, url): url for url in bonbanh_urls}
    for future in as_completed(futures):
        url = futures[future]
        try:
            features = future.result()
            if features:
                bonbanh_data.append(features)
        except Exception as e:
            print(f"Lỗi với URL Bonbanh {url}: {e}")

df_bonbanh = pd.DataFrame(bonbanh_data)
df_bonbanh.to_csv('../datasets/raw_bonbanh_car_features.csv', index=False)
print(f"Bonbanh: Đã lưu {len(bonbanh_data)} bản ghi vào datasets/raw_bonbanh_car_features.csv")

Bắt đầu trích xuất dữ liệu Chotot...
Chotot: Đã lưu 100 bản ghi vào datasets/raw_chotot_car_features.csv
Bắt đầu trích xuất dữ liệu Bonbanh...
Chotot: Đã lưu 100 bản ghi vào datasets/raw_chotot_car_features.csv
Bắt đầu trích xuất dữ liệu Bonbanh...
Bonbanh: Đã lưu 100 bản ghi vào datasets/raw_bonbanh_car_features.csv
Bonbanh: Đã lưu 100 bản ghi vào datasets/raw_bonbanh_car_features.csv


## Kết quả
- **Chotot**: Đã lưu dữ liệu vào `datasets/raw_chotot_car_features.csv`, mỗi bản ghi bao gồm tiêu đề, giá, các thông số kỹ thuật, vị trí, người bán và mô tả.
- **Bonbanh**: Đã lưu dữ liệu vào `datasets/raw_bonbanh_car_features.csv`, giữ nguyên tiêu đề gốc và các thuộc tính cốt lõi đã được lọc.
- **Tối ưu hóa**: Nhờ sử dụng đa luồng, thời gian trích xuất giảm đáng kể so với xử lý tuần tự. Với 100 URL, thời gian giảm từ ~150 giây xuống còn khoảng 15-20 giây (tùy thuộc tốc độ mạng và server).